<div class='bar_title'></div>

*Data Driven Decisions in Practice (D3IP): Urban Analytics*

# Case Study: Predicting AirBnB Accomodation Prices

Gunther Gust & Nikolai Stein

Data Driven Decisions (D3) Group <br>
Center for Artificial Intelligence & Data Science <br>



<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

Complete the case study outlined by the steps below. Remember to always comment your code and document your findings so that your notebook is easy to read and follow! (Apart from correctness, the style of the notebook will also affect your grade!)



# PART 1: Data Loading and Exploratory Data Analysis

Insructions:

* Load the `airbnb.geojson` file into this notebook as a geodataframe
* Explore the content of each column of the geodataframe using methods of your choice (descriptive statistics, plots etc.). Describe your findings in the markdown cells.
* Create a plot that displays the location of the airbnb listings and the price. Add a basemap of San Diego to the plot.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt 
import seaborn as sns
import contextily as cx

In [ ]:
# Load the geodataframe <insert your code here>....
db = gpd.read_file("airbnb_listings.geojson")

#exploring the dataset
f, ax = plt.subplots(1, figsize=(16, 16))
db.plot(ax = ax, column = 'price', legend = True, alpha = 0.5, cmap = 'inferno', markersize = 10)
cx.add_basemap(ax, crs = db.crs, source=cx.providers.Esri.WorldImagery)
plt.show()
db.describe()

#Beobachtungen: 
#- sehr viele sehr günstige AirBnBs (75% kosten bis zu 250$), ein paar wenige Ausreißer die extrem teuer sind (max 4900$)
#- Preisbereich obere 25% liegen auffällig nahe an den Küstenregionen / Strandregionen
#- rechter Cluster ist ringförmig (Balboa Park in der Mitte mit Zoo)
#- La Jolla hat besonders viele AirBnBs aus den oberen 25%

These are the variables you should analyze and later use to predict the `price`:

In [ ]:

variable_names = [
    "accommodates",  # Number of people it accommodates
    "bathrooms",  # Number of bathrooms
    "bedrooms",  # Number of bedrooms
    "beds",  # Number of beds
    # Below are binary variables, 1 True, 0 False
    "rt_Private_room",  # Room type: private room
    "rt_Shared_room",  # Room type: shared room
    "pg_Condominium",  # Property group: condo
    "pg_House",  # Property group: house
    "pg_Other",  # Property group: other
    "pg_Townhouse",  # Property group: townhouse
]

# PART 2 Feature Engineering: Get points-of-interest (POIs) and prepare them

Instructions:

* Use the method `features_from_address()` of osmnx to download POIs of the type `amenity` (select the types of amenities to include from this list here: https://wiki.openstreetmap.org/wiki/Key:amenity)
* Pay attention to set the `dist` parameter to an appropiate value 

In [ ]:
import osmnx as ox

In [ ]:
# Download the POI data <your code here> ...
import osmnx as ox
#for plotting
pois = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : ["bar", "atm", "cafe", "restaurant"]})

#for KDE
poiBars = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "bar"})
poiATMs = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "atm"})
poiCafes = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "cafe"})
poiRestaurants = ox.features_from_address("San Diego, USA", dist = 5000, tags = {"amenity" : "restaurant"})


In [ ]:
#poiBars.head()
pois.head()

Data cleaning: The resulting `pois` geodataframe may have a composite row index. In addition, some POIs may be of a strange element_type. You can use the following code to eliminate these issues.

In [ ]:
# Clean the POI data
pois.reset_index(inplace=True) # reset the index of the data frame
pois = pois[pois.element_type=="node"] # eliminate all POIs that are not of the type "node"
#pois.head() # check the results

#Clean POI bei amenity
#Bar Point of Interests
poiBars.reset_index(inplace=True)
poiBars = poiBars[poiBars.element_type == "node"]
#poiBars.head()

#ATM Point of Interests
poiATMs.reset_index(inplace=True)
poiATMs = poiATMs[poiATMs.element_type == "node"]
#poiATMs.head()

#Cafe Point of Interests
poiCafes.reset_index(inplace=True)
poiCafes = poiCafes[poiCafes.element_type == "node"]
#poiCafes.head()

#Restaurant Point of Interests
poiRestaurants.reset_index(inplace=True)
poiRestaurants = poiRestaurants[poiRestaurants.element_type == "node"]
#poiRestaurants.head()

Instructions:
* Plot the POIs spatially (use again a background map of San Diego)
* When you use POIs of different amenity types, color the POIs differently

In [ ]:
# <Your code here>
#pois.plot(alpha = 0.1)
# Setup figure and axis
#f, ax = plt.subplots(1)
# Plot layer of polygons on the axis
#ax = pois.plot(alpha = 0.1, ax=ax)
# Remove axis frames
#ax.set_axis_off()
# Display
#plt.show()


#subplots per POI by amenity
#Bars
#fBars, axBars = plt.subplots(1)
#axBars = poiBars.plot(alpha = 0.1, ax=axBars)
#axBars.set_axis_off()
#plt.show()

#ATMs
#fATMs, axATMs = plt.subplots(1)
#axATMs = poiATMs.plot(alpha = 0.1, ax=axATMs, color = 'brown')
#axATMs.set_axis_off()
#plt.show()

#Cafes
#fCafes, axCafes = plt.subplots(1)
#axCafes = poiCafes.plot(alpha = 0.1, ax=axCafes, color = 'orange')
#axCafes.set_axis_off()
#plt.show()

#Restaurants
#fRestaurants, axRestaurants = plt.subplots(1)
#axRestaurants = poiRestaurants.plot(alpha = 0.1, ax=axRestaurants, color = 'green')
#axRestaurants.set_axis_off()
#plt.show()

#sanDiego = db.loc[[12], :].to_crs(epsg=4326)

#ORIGINAL LÖSUNG#
fAllPOIs, ax = plt.subplots(1)
#poiBars.plot(alpha = 0.5, ax = ax, edgecolor = 'black')
#poiATMs.plot(alpha = 0.3, ax = ax, color = 'purple', edgecolor = 'black')
#poiCafes.plot(alpha = 0.5, ax = ax, color = 'orange', edgecolor = 'black')
#poiRestaurants.plot(alpha = 0.15, ax = ax, color = 'black', edgecolor = 'black')
#ORIGINAL LÖSUNG#

#poiBars.plot(alpha = 0.3, ax = ax, column = 'amenity', legend = True)
#poiATMs.plot(alpha = 0.25, ax = ax, column = 'amenity', legend = True)
#poiCafes.plot(alpha = 0.3, ax = ax, column = 'amenity', legend = True)
#poiRestaurants.plot(alpha = 0.2, ax = ax, column = 'amenity', legend = True)

#poiBars.plot(ax = ax, column = 'amenity', legend = True, alpha = 0.5)
#poiATMs.plot(ax = ax, column = 'amenity', legend = True, alpha = 0.5)
#poiCafes.plot(ax = ax, column = 'amenity', legend = True, alpha = 0.5)
#poiRestaurants.plot(ax = ax, column = 'amenity', legend = True, alpha = 0.5)


#ORIGINAL LÖSUNG#
#cx.add_basemap(ax, crs = db.crs)

#fAllPOIs.suptitle("Color code for POIs of different types of amenity in San Diego")
#ax.set_title("Blue = Bars | Purple = ATMs | Orange = Cafes | Black = Restaurants")
#ax.set_axis_off()

#plt.show()
#ORIGINAL LÖSUNG#

pois.plot(alpha = 1, ax = ax, edgecolor = 'black', column = 'amenity', legend = True, cmap = 'plasma')
cx.add_basemap(ax, crs = db.crs)
ax.set_title("Amenities in San Diego")
ax.set_axis_off()
plt.show()




For each POI amenity type, create a kernel density estimation (KDE):
* Convert the `geometry` of the POI into a suitable coordinate data format (you may use the provided function `create_coordinate_array` for this)
* Feed the resulting coordinates into the `gaussian_kde` function and estimate the function
* Also convert the `geometry` of the Airbnb listings into the coordinate data format (you may use the provided function `create_coordinate_array` for this)
* Using the converted Airbnb geometries, compute the KDE for the locations of the Airbnb listings
* Add the KDE estimate as additional columns to your original airbnb geodataframe

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
# helper function to convert the geometries into a suitable coordinate format for the KDE
def create_coordinate_array(geometries): 
    x_values = []
    y_values = []

# Iterate through each row in the GeoDataFrame
    for multipoint in geometries:
        # Ensure the geometry is indeed MultiPoint; if it's just a single Point, wrap it in a list
        points = list(multipoint.geoms) if hasattr(multipoint, "geoms") else [multipoint]
        
        # For each Point in the MultiPoint, extract x and y values
        for point in points:
            x_values.append(point.x)
            y_values.append(point.y)

    # Optionally, convert the lists to numpy arrays for further processing
    x_values = np.array(x_values)
    y_values = np.array(y_values)

    # Rearrange data to create a 2D array of x and y coordinates
    xy = np.vstack([x_values,y_values])

    return xy


# Example Usage for the Airbnb geodataframe
#airbnb_array = create_coordinate_array(airbnb.geometry)

In [ ]:
#<Your code here>

#pro POI amenity type eine KDE durchführen und an Original Geodataframe anhängen als Spalte


# Rearrange data to create a 2D array of x and y coordinates
#xy = np.vstack([x_values,y_values])

#POI Bar
xyBars = create_coordinate_array(poiBars.geometry)

# Compute the density estimation
kde = gaussian_kde(xyBars)

# Define the grid points where you want to calculate the density
xmin, xmax = xyBars[0].min(), xyBars[0].max()
ymin, ymax = xyBars[1].min(), xyBars[1].max()

xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j] #generate 100 evenly spaced points between min and max

# Evaluate the density at grid points
density = kde(np.vstack([xx.ravel(), yy.ravel()]))
density = density.reshape(xx.shape) # reshape to the original shape of xx (for plotting)

#POI ATMs
xyATMs = create_coordinate_array(poiBars.geometry)

# Compute the density estimation
kde = gaussian_kde(xyATMs)

# Define the grid points where you want to calculate the density
xmin, xmax = xyATMs[0].min(), xyATMs[0].max()
ymin, ymax = xyATMs[1].min(), xyATMs[1].max()

xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j] #generate 100 evenly spaced points between min and max

# Evaluate the density at grid points
density = kde(np.vstack([xx.ravel(), yy.ravel()]))
density = density.reshape(xx.shape) # reshape to the original shape of xx (for plotting)



Create a spatial point plot(s) of the Airbnb listings and color the points according to the KDE estimates, in order to check your results.

In [ ]:
# <your code here>
plt.imshow(np.rot90(density), cmap='hot', extent=[xmin, xmax, ymin, ymax])
plt.colorbar(label='Density')
plt.scatter(xyBars[0], xyBars[1], s=10, c='blue', alpha=0.5)
plt.title('POIs')
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
plt.show()

# PART 3: Modeling and Analysis (aka Machine Learning)

Use your dataset generated in the previous steps to predict AirBnb prices

Instructions:

* Split your data set into training and validation data sets
* Define an error metric (or several)
* Train at least one machine learning model (e.g. random forest) 
* Tune the hyperparameters (if applicable for the model)
* Evaluate the accuracy of the predicted prices against actual prices
* Compare the performance of the previous models when using different input data sets (benchmarks). Make sure to include the naive benchmark of predicting always the mean price.

In [ ]:
from sklearn.model_selection import train_test_split

X = ""
y = ""

## Model 1: Trivial Baseline

In [ ]:
# Trivial baseline: Predict always the mean price 
# <your code here>

## Model 2: ...

In [ ]:
# <your code here>

## Model 3: ...